**Ana Carolina Prado Ricciardi**

Prof. Francisco Aparecido Rodrigues, PhD

SME5924 - Processos Dinâmicos em Redes Complexas (2025)



**Referência** :
Pastor-Satorras, R., Castellano, C., Van Mieghem, P., & Vespignani, A. (2015).
Epidemic processes in complex networks. Reviews of Modern Physics, 87(3), 925–979.
https://doi.org/10.1103/RevModPhys.87.925



## Introdução

Neste estudo, buscamos compreender **como o distanciamento social afeta a propagação de epidemias**, por meio da simulação computacional de **modelos epidemiológicos em redes complexas**. A abordagem baseia-se na hipótese de que **a estrutura das interações sociais influencia diretamente o alcance e a velocidade de disseminação de uma doença infecciosa**. Para isso, utilizamos a biblioteca [EoN – Epidemics on Networks](https://epidemicsonnetworks.readthedocs.io/en/latest/), que permite a simulação precisa de modelos como **SIR** (Susceptível-Infectado-Recuperado) e **SIS** (Susceptível-Infectado-Suscetível) sobre grafos.

O foco principal está na análise dos efeitos da **remoção de conexões na rede** – uma analogia ao **distanciamento social implementado como política de saúde pública**. Através dessa remoção de arestas, buscamos investigar como diferentes estratégias de intervenção impactam o curso da epidemia, especialmente no que diz respeito à **fração de indivíduos recuperados** ao final da propagação.

### Objetivos

- Implementar modelos de propagação de epidemias (SIR e SIS) sobre diferentes redes complexas;
- Simular o efeito da **remoção aleatória de conexões**, representando distanciamento social não direcionado;
- Avaliar estratégias **direcionadas** de distanciamento, com remoção de links baseada em **centralidade de arestas**:
  - **Betweenness centrality** (intermediação)
  - **Grau médio dos vértices conectados**
- Comparar o comportamento da epidemia em três topologias clássicas de rede:
  - **Erdős–Rényi (ER)** – rede aleatória
  - **Barabási–Albert (BA)** – rede com hubs
  - **Watts–Strogatz (WS)** – rede de mundo pequeno
- Analisar uma **rede empírica real** (Advogato), representando interações sociais em uma comunidade online.

### Metodologia

As simulações seguem as seguintes etapas:

1. Geração de redes artificiais e carregamento da rede real (Advogato);
2. Execução dos modelos SIR e SIS com diferentes parâmetros epidemiológicos;
3. Cálculo da **fração de infectados e recuperados ao longo do tempo**;
4. Aplicação de estratégias de remoção de links (aleatória e orientada por centralidade);
5. Comparação quantitativa do impacto sobre a **propagação e o alcance da epidemia**.

A métrica principal utilizada ao longo deste estudo é a **fração final de nós recuperados** (no modelo SIR), que representa a **proporção da população que adoeceu e se recuperou**, servindo como proxy do impacto da epidemia.

---

Este notebook está organizado de forma modular, permitindo a reprodutibilidade dos experimentos e a análise comparativa entre topologias de rede e estratégias de intervenção.


In [ ]:
!pip install EoN

In [ ]:

# 🔹 Seção 1: Importação de Bibliotecas
import networkx as nx  # para criação e manipulação de redes
import EoN  # biblioteca para simulação de epidemias em redes
import matplotlib.pyplot as plt  # para visualizações
import numpy as np  # operações numéricas
import pandas as pd  # estrutura de dados
import random  # controle de aleatoriedade
from tqdm import tqdm  # barra de progresso
from scipy.io import mmread  # leitura de arquivos .mtx

# Definindo uma semente para reprodutibilidade dos resultados
random.seed(42)
np.random.seed(42)

##Contextualização

Modelos epidemiológicos são fundamentais para compreender como doenças se espalham em uma população. Neste trabalho, representamos a população como uma rede complexa, em que os nós representam indivíduos e as arestas representam conexões sociais ou interações pelas quais a doença pode ser transmitida.

Abaixo, iniciamos a simulação com a construção de uma rede real baseada em dados empíricos (rede de confiança social), que pode ser substituída por qualquer outro dataset de arestas.

In [ ]:
# Parâmetros da rede
N = 100                # número de nós (indivíduos)
av_degree = 8          # grau médio de conexões por nó
p = float(av_degree)/float(N)  # probabilidade de conexão no modelo ER
m = int(av_degree/2)   # número de conexões adicionadas por novo nó no modelo BA
kappa = av_degree       # número de vizinhos no modelo WS

# A rede é carregada a partir de um arquivo com lista de arestas.
# Cada linha do arquivo contém dois nós conectados por uma aresta e um peso associado.
G = nx.read_edgelist("/content/advogato.txt", nodetype=int, data=[("weiht", float)])

# Garantir que a rede seja não direcionada (bidirecional)
G = G.to_undirected()

# Ordenar os componentes conexos da rede, do maior para o menor
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

# Seleciona o maior componente conexo
G = G.subgraph(Gcc[0])

# Converte os rótulos dos nós para inteiros consecutivos a partir do zero
G = nx.convert_node_labels_to_integers(G, first_label=0)


In [ ]:
print(f"Número de nós no maior componente conexo: {G.number_of_nodes()}")
print(f"Número de arestas: {G.number_of_edges()}")
graus = [G.degree(n) for n in G.nodes()]
print(f"Grau médio: {np.mean(graus):.2f}")


In [ ]:
def momment_of_degree_distribution(G, m):
    """
    Calcula o m-ésimo momento da distribuição de graus de uma rede G.

    Parâmetros:
    - G: objeto networkx.Graph
    - m: ordem do momento a ser calculado

    Retorna:
    - Valor médio do grau elevado à potência m
    """
    M = 0
    N = len(G)
    for i in G.nodes():
        M = M + G.degree(i)**m
    M = M / N
    return M

# Exemplo de uso:
print(f"Momento de ordem 1 (grau médio): {momment_of_degree_distribution(G, 1):.2f}")
print(f"Momento de ordem 2: {momment_of_degree_distribution(G, 2):.2f}")


#Simulação SIR a partir de um único nó inicial (seed node)
## Objetivo

Realizar a simulação da propagação de uma epidemia utilizando o modelo SIR (Susceptível–Infectado–Recuperado) sobre a rede G (definida anteriormente), iniciando a infecção a partir de um único nó (paciente zero). Essa simulação adota o modelo reativo, onde cada nó infectado tenta transmitir a doença a todos os seus vizinhos em cada passo de tempo.

## 1.1 Dinâmica do Modelo SIR

O modelo SIR é um dos modelos epidemiológicos clássicos que divide a população em três compartimentos:

S (Susceptíveis): indivíduos que ainda não foram infectados, mas podem ser;

I (Infectados): indivíduos atualmente infectados e capazes de transmitir a doença;

R (Recuperados): indivíduos que se recuperaram e não transmitem mais.

A dinâmica é regida por dois parâmetros:

𝛽
β: taxa de infecção por contato entre um indivíduo infectado e um suscetível;

𝜇
μ: taxa de recuperação dos indivíduos infectados.

A evolução do sistema se dá por meio das transições:

𝑆
→
𝛽
𝐼
S
β
​
 I: quando um suscetível entra em contato com um infectado;

𝐼
→
𝜇
𝑅
I
μ
​
 R: quando um infectado se recupera.

In [ ]:
# Função para simular a dinâmica SIR iniciando de um único nó infectado (seed)
def SIR_single_seed(G, seed, beta=0.3, mu=1):
    def find(v, i):
        """
        Função auxiliar para localizar as posições onde o vetor v possui o valor i.
        Usado para identificar os nós em determinado estado (S, I ou R).
        """
        l = []
        pos = 0
        for x in v:
            if x == i:
                l.append(pos)
            pos += 1
        return l

    # Processo reativo: dinâmica SIR
    seed_node = seed  # nó inicial infectado (paciente zero)
    # Use the actual number of nodes in graph G to initialize the vector_states
    vector_states = np.zeros(G.number_of_nodes())  # vetor que armazena o estado de cada nó (0=S, 1=I, 2=R)
    vector_states[seed_node] = 1  # infecta o nó inicial
    ninfected = 1                # contador de nós infectados
    t = 0                        # tempo inicial

    # Listas para armazenar a evolução temporal dos compartimentos
    infected = list()    # lista dos nós infectados a cada passo de tempo
    vt = list()          # armazena os tempos (t)
    vI = list()          # fração de infectados ao longo do tempo
    vR = list()          # fração de recuperados ao longo do tempo
    vS = list()          # fração de suscetíveis ao longo do tempo

    # Simulação principal do modelo SIR
    while ninfected > 0:  # continua enquanto houver ao menos um infectado
        infected = find(vector_states, 1)  # identifica os nós atualmente infectados
        for i in infected:
            neigs = G.neighbors(i)  # obtém os vizinhos do infectado
            for j in neigs:
                if np.random.rand() < beta:  # com probabilidade beta, ocorre a infecção
                    if vector_states[j] != 2:  # se o vizinho ainda não estiver recuperado
                        vector_states[j] = 1   # o vizinho se torna infectado

        # Após a tentativa de infecção, os infectados podem se recuperar
        for k in infected:
            if np.random.rand() < mu:  # com probabilidade mu, ocorre recuperação
                vector_states[k] = 2   # o nó passa para o estado de recuperado

        # Atualização do número de infectados
        ninfected = len(find(vector_states, 1))

        # Armazena a fração de cada compartimento no tempo t, using the correct total number of nodes
        vI.append(ninfected / G.number_of_nodes())
        vR.append(len(find(vector_states, 2)) / G.number_of_nodes())
        vS.append(len(find(vector_states, 0)) / G.number_of_nodes())
        t += 1
        vt.append(t)

    return vI, vS, vR, vt


Modelo Reativo (Reactive process): Cada nó infectado entra em contato com todos os seus vizinhos em cada passo de tempo e tenta infectá-los com probabilidade
β.

A simulação termina quando não restam mais infectados na rede.

As listas vI, vS e vR retornam a fração de indivíduos em cada estado ao longo do tempo, útil para visualização e análise posterior.


In [ ]:
# Parâmetros epidemiológicos
beta = 0.5  # probabilidade de infecção por contato
mu = 0.7    # probabilidade de recuperação
seed = 0    # nó inicial infectado

# Executa a simulação da dinâmica SIR a partir de um único seed
vI, vS, vR, vt = SIR_single_seed(G, seed, beta, mu)


In [ ]:
# Gráfico da evolução temporal das frações de indivíduos em cada estado
plt.figure(figsize=(10,5))
plt.plot(vt, vI, 'ro--', label='Infectados (I)')
plt.plot(vt, vR, 'bo--', label='Recuperados (R)')
plt.plot(vt, vS, 'go--', label='Suscetíveis (S)')
plt.xlabel("Tempo (t)", fontsize=15)
plt.ylabel("Fração de nós", fontsize=15)
plt.title("Dinâmica SIR iniciando do nó 0", fontsize=14)
plt.legend()
plt.grid(True)
plt.show()


## Dinâmica do Modelo SIS

O modelo SIS é adequado para descrever doenças em que não há imunidade permanente após a infecção (como gripes comuns ou doenças sexualmente transmissíveis em estágios crônicos). A dinâmica é dada por dois estados:

S (Susceptível): indivíduo saudável, mas vulnerável à infecção;

I (Infectado): indivíduo atualmente infectado e transmissor.

As transições são:

𝑆
→
𝛽
𝐼
S
β
​
 I: infecção por contato com infectados;

𝐼
→
𝜇
𝑆
I
μ
​
 S: recuperação sem imunidade (retorna ao estado suscetível).

 ## Implementação da Dinâmica SIS com tempo máximo

Como o modelo não termina naturalmente (não há estado absorvente), precisamos definir um número máximo de passos de tempo.

In [ ]:
# Função para simular a dinâmica SIS iniciando de um único nó infectado (seed)
def SIS_single_seed(G, seed, beta=0.3, mu=1, Tmax=50):
    def find(v, i):
        """
        Função auxiliar que retorna as posições (índices) no vetor v que contêm o valor i.
        Utilizada para identificar os nós que estão em um determinado estado (infectado, por exemplo).
        """
        l = []
        pos = 0
        for x in v:
            if x == i:
                l.append(pos)
            pos += 1
        return l

    # Processo reativo: dinâmica SIS
    seed_node = seed  # nó semente (primeiro infectado) - can be adjusted outside the function
    # Use the actual number of nodes in graph G to initialize the vector_states
    vector_states = np.zeros(G.number_of_nodes())  # vetor de estados: 0 = suscetível, 1 = infectado
    vector_states[seed_node] = 1  # infecta o nó inicial
    ninfected = 1  # número inicial de infectados

    # Listas para armazenar a evolução dos estados ao longo do tempo
    infected = list()
    vt = list()  # tempo
    vI = list()  # fração de infectados
    vS = list()  # fração de suscetíveis

    # Simulação do modelo até o tempo máximo definido (Tmax)
    for t in np.arange(0, Tmax):
        infected = find(vector_states, 1)  # identifica os nós infectados
        for i in infected:  # cada infectado tenta infectar seus vizinhos
            neigs = G.neighbors(i)
            for j in neigs:
                if np.random.rand() < beta:
                    vector_states[j] = 1  # vizinho se torna infectado com probabilidade beta

        # Cada infectado pode se recuperar e voltar ao estado suscetível
        for k in infected:
            if np.random.rand() < mu:
                vector_states[k] = 0  # no modelo SIS, o nó recuperado volta a ser suscetível

        # Atualização das métricas de interesse
        # Use the actual number of nodes in graph G for calculating fractions
        ninfected = len(find(vector_states, 1))
        vI.append(ninfected / G.number_of_nodes())  # fração de infectados
        vS.append(len(find(vector_states, 0)) / G.number_of_nodes())  # fração de suscetíveis
        vt.append(t)

    return vS, vI, vt

O modelo SIS representa doenças que não geram imunidade permanente;

Ao contrário do SIR, os infectados retornam ao estado de suscetível após a recuperação;

Como não há estado absorvente, é necessário definir um tempo máximo de simulação (Tmax);

As curvas de infectados e suscetíveis tendem a atingir um estado estacionário.

# Simulação com Múltiplos Nós Infectados

Até agora, a infecção foi iniciada com apenas um indivíduo (seed node). Nesta etapa, simularemos a dinâmica da epidemia considerando que diversos indivíduos estão infectados no início, o que pode ocorrer em surtos com múltiplos focos ou quando há exposição simultânea.

In [ ]:
# Geração de diferentes tipos de redes complexas

# Parâmetros comuns para todos os modelos
N = 100               # número de nós
av_degree = 8         # grau médio
p = float(av_degree) / float(N)  # probabilidade de conexão no modelo ER
m = int(av_degree / 2)           # número de arestas por novo nó no modelo BA
kappa = av_degree     # número de vizinhos no modelo WS

# Geração das redes
G_ER = nx.erdos_renyi_graph(N, p)                     # Erdős–Rényi
G_BA = nx.barabasi_albert_graph(N, m)                # Barabási–Albert
G_WS = nx.watts_strogatz_graph(N, kappa, 0.1)        # Watts–Strogatz



In [ ]:
# Layout fixo para manter consistência entre os gráficos
pos_er = nx.spring_layout(G_ER, seed=42)
pos_ba = nx.spring_layout(G_BA, seed=42)
pos_ws = nx.spring_layout(G_WS, seed=42)

# Figura com 3 subgráficos
plt.figure(figsize=(18, 5))

# Erdős–Rényi
plt.subplot(1, 3, 1)
nx.draw(G_ER, pos=pos_er, node_size=30, edge_color='gray', with_labels=False)
plt.title("Rede Aleatória Erdős–Rényi (ER)")

# Barabási–Albert
plt.subplot(1, 3, 2)
nx.draw(G_BA, pos=pos_ba, node_size=30, edge_color='gray', with_labels=False)
plt.title("Rede com Hubs Barabási–Albert (BA)")

# Watts–Strogatz
plt.subplot(1, 3, 3)
nx.draw(G_WS, pos=pos_ws, node_size=30, edge_color='gray', with_labels=False)
plt.title("Rede de Mundo Pequeno Watts–Strogatz (WS)")

plt.suptitle("Visualização Comparativa das Redes Complexas", fontsize=16)
plt.tight_layout()
plt.show()


## Dinâmica SIS Média com 20 Seeds Aleatórios

Executar a simulação SIS com 20 nós aleatórios como seeds iniciais em cada rede (ER, BA, WS), e calcular a fração média de infectados ao longo do tempo. Isso reduz significativamente o tempo de execução e mantém a qualidade da comparação.



In [ ]:
from random import sample

# Parâmetros epidemiológicos
beta = 0.2
mu = 0.5
Tmax = 50
num_seeds = 20


In [ ]:
#Simulação para as 3 Redes

# Inicialização
av_I_er = np.zeros(Tmax)
av_I_ba = np.zeros(Tmax)
av_I_ws = np.zeros(Tmax)

# Erdős–Rényi
sample_seeds_er = sample(list(G_ER.nodes()), num_seeds)
for seed_node in sample_seeds_er:
    vS, vI, vt = SIS_single_seed(G_ER, seed_node, beta, mu, Tmax)
    for x in range(len(vI)):
        av_I_er[x] += vI[x]
av_I_er /= num_seeds

# Barabási–Albert
sample_seeds_ba = sample(list(G_BA.nodes()), num_seeds)
for seed_node in sample_seeds_ba:
    vS, vI, vt = SIS_single_seed(G_BA, seed_node, beta, mu, Tmax)
    for x in range(len(vI)):
        av_I_ba[x] += vI[x]
av_I_ba /= num_seeds

# Watts–Strogatz
sample_seeds_ws = sample(list(G_WS.nodes()), num_seeds)
for seed_node in sample_seeds_ws:
    vS, vI, vt = SIS_single_seed(G_WS, seed_node, beta, mu, Tmax)
    for x in range(len(vI)):
        av_I_ws[x] += vI[x]
av_I_ws /= num_seeds

# Eixo de tempo
vt = np.arange(0, Tmax)


In [ ]:
#Visualização Comparativa – 20 Seeds Aleatórios

plt.figure(figsize=(10,6))
plt.plot(vt, av_I_er, 'r-', label='ER (Erdős–Rényi)')
plt.plot(vt, av_I_ba, 'b-', label='BA (Barabási–Albert)')
plt.plot(vt, av_I_ws, 'g-', label='WS (Watts–Strogatz)')
plt.xlabel("Tempo (t)", fontsize=14)
plt.ylabel("Fração média de infectados", fontsize=14)
plt.title("Dinâmica SIS Média com 20 Seeds Aleatórios", fontsize=15)
plt.legend()
plt.grid(True)
plt.show()



BA (Barabási–Albert): mostra maior persistência da infecção, mesmo com poucos seeds, devido aos hubs.

ER: curva moderada, com crescimento rápido mas sem sustentação prolongada.

WS: comportamento intermediário, com propagação local e estabilidade.

A escolha da topologia influencia diretamente o alcance e a duração média da epidemia, mesmo com um número reduzido de seeds.

#Efeito da Remoção de Conexões (Distanciamento Social)
 Objetivo
Simular a propagação do modelo SIR sobre uma rede antes e depois da remoção de uma fração das conexões. Essa remoção pode ser:

Aleatória: simula distanciamento social generalizado, sem foco estratégico;

Orientada (centralidade): simula políticas direcionadas a interromper contatos mais influentes (hubs, pontes, etc.).

Aqui, vamos começar com a remoção aleatória de links, e analisar seu efeito na fração final de recuperados, que representa o total de pessoas que adoeceram ao longo da epidemia.

In [ ]:
#Função para simular com remoção aleatória

def SIR_with_random_removal(G_original, removal_fraction, beta=0.3, mu=0.1, Tmax=50, num_seeds=10):
    G = G_original.copy()
    num_edges_to_remove = int(removal_fraction * G.number_of_edges())
    # Convert G.edges() to a list before sampling
    edges_to_remove = random.sample(list(G.edges()), num_edges_to_remove)
    G.remove_edges_from(edges_to_remove)

    R_final = []
    seeds = sample(list(G.nodes()), num_seeds)
    for seed in seeds:
        vI, vS, vR, vt = SIR_single_seed(G, seed, beta, mu)
        R_final.append(vR[-1])

    return np.mean(R_final)


In [ ]:
#Varredura: 0% a 50% de remoção de links

removal_rates = np.linspace(0, 0.5, 11)  # de 0% a 50% em etapas de 5%

R_er = []
R_ba = []
R_ws = []

for r in removal_rates:
    R_er.append(SIR_with_random_removal(G_ER, r, beta=0.3, mu=0.1, Tmax=50, num_seeds=10))
    R_ba.append(SIR_with_random_removal(G_BA, r, beta=0.3, mu=0.1, Tmax=50, num_seeds=10))
    R_ws.append(SIR_with_random_removal(G_WS, r, beta=0.3, mu=0.1, Tmax=50, num_seeds=10))


In [ ]:
# Gráfico: Fração de Recuperados vs. Remoção de Links

plt.figure(figsize=(10,6))
plt.plot(removal_rates * 100, R_er, 'o-', label='ER (Erdős–Rényi)')
plt.plot(removal_rates * 100, R_ba, 's-', label='BA (Barabási–Albert)')
plt.plot(removal_rates * 100, R_ws, '^-', label='WS (Watts–Strogatz)')
plt.xlabel("Percentual de links removidos (%)", fontsize=13)
plt.ylabel("Fração média de recuperados", fontsize=13)
plt.title("Efeito da Remoção Aleatória de Links nas 3 Redes (SIR)", fontsize=15)
plt.legend()
plt.grid(True)
plt.show()



##  Interpretação Comparativa
BA (com hubs): mostra maior vulnerabilidade à remoção aleatória — poucos cortes já afetam hubs críticos.

WS: afeta moderadamente a propagação, pois combina clustering local com atalhos.

ER: tende a reduzir a propagação de forma mais gradual e linear.

Mesmo sem estratégia, a remoção aleatória de arestas já causa forte redução no impacto epidêmico, especialmente em redes com alto grau de heterogeneidade (como a BA).

# Remoção Direcionada de Conexões com Base em Centralidade
Objetivo
Avaliar o impacto da remoção de links com base em medidas de centralidade sobre a propagação de uma epidemia SIR. Essa simulação representa intervenções estratégicas, onde são interrompidas as conexões que mais contribuem para a disseminação, segundo critérios estruturais:

Betweenness centrality (intermediação): remove arestas mais usadas como “pontes”;



In [ ]:
#Funções auxiliares para ordenação e remoção

def edge_betweenness_sorted(G):
    """Retorna lista de arestas ordenadas por betweenness centrality (decrescente)."""
    bw = nx.edge_betweenness_centrality(G)
    return sorted(bw, key=bw.get, reverse=True)

def edge_avg_degree_sorted(G):
    """Retorna lista de arestas ordenadas por grau médio dos nós que conecta (decrescente)."""
    edge_score = {}
    for u, v in G.edges():
        deg_avg = (G.degree(u) + G.degree(v)) / 2
        edge_score[(u, v)] = deg_avg
    return sorted(edge_score, key=edge_score.get, reverse=True)


In [ ]:
# Função de simulação SIR com remoção direcionada
def SIR_with_targeted_removal(G_original, removal_fraction, strategy='betweenness', beta=0.3, mu=0.1, Tmax=50, num_seeds=10):
    G = G_original.copy()
    num_edges_to_remove = int(removal_fraction * G.number_of_edges())

    if strategy == 'betweenness':
        sorted_edges = edge_betweenness_sorted(G)
    elif strategy == 'avg_degree':
        sorted_edges = edge_avg_degree_sorted(G)
    else:
        raise ValueError("Estratégia não reconhecida. Use 'betweenness' ou 'avg_degree'.")

    # Remove as arestas mais centrais
    edges_to_remove = sorted_edges[:num_edges_to_remove]
    G.remove_edges_from(edges_to_remove)

    # Executa simulação SIR
    R_final = []
    seeds = sample(list(G.nodes()), num_seeds)
    for seed in seeds:
        vI, vS, vR, vt = SIR_single_seed(G, seed, beta, mu)
        R_final.append(vR[-1])

    return np.mean(R_final)


In [ ]:
#Simulação para diferentes taxas de remoção

removal_rates = np.linspace(0, 0.5, 11)

# Inicialização de dicionários para armazenar os resultados
results = {
    'ER': {'random': [], 'betweenness': [], 'avg_degree': []},
    'BA': {'random': [], 'betweenness': [], 'avg_degree': []},
    'WS': {'random': [], 'betweenness': [], 'avg_degree': []}
}

# Função para iterar por estratégia e rede
for r in removal_rates:
    for net_name, G_net in zip(['ER', 'BA', 'WS'], [G_ER, G_BA, G_WS]):
        results[net_name]['random'].append(
            SIR_with_random_removal(G_net, r, beta=0.3, mu=0.1, Tmax=50, num_seeds=10))
        results[net_name]['betweenness'].append(
            SIR_with_targeted_removal(G_net, r, strategy='betweenness', beta=0.3, mu=0.1, Tmax=50, num_seeds=10))
        results[net_name]['avg_degree'].append(
            SIR_with_targeted_removal(G_net, r, strategy='avg_degree', beta=0.3, mu=0.1, Tmax=50, num_seeds=10))


In [ ]:
#Gráfico: Fração de Recuperados vs. Porcentagem de Remoção (Direcionada)
plt.figure(figsize=(18,5))

for i, net_name in enumerate(['ER', 'BA', 'WS']):
    plt.subplot(1, 3, i+1)
    plt.plot(removal_rates*100, results[net_name]['random'], 'o-', label='Aleatória')
    plt.plot(removal_rates*100, results[net_name]['betweenness'], 's-', label='Betweenness')
    plt.plot(removal_rates*100, results[net_name]['avg_degree'], '^-', label='Grau médio')
    plt.xlabel("Links removidos (%)", fontsize=12)
    plt.ylabel("Fração média de recuperados", fontsize=12)
    plt.title(f"Rede {net_name}", fontsize=14)
    plt.grid(True)
    plt.legend()

plt.suptitle("Comparação de Estratégias de Remoção de Links (Modelo SIR)", fontsize=16)
plt.tight_layout()
plt.show()


A remoção orientada por centralidade (especialmente betweenness) é claramente mais eficaz para conter a propagação epidêmica em redes reais, especialmente quando há hubs ou pontes estruturais.

# Interpretação Final: Impacto da Remoção de Links no Modelo SIR
## Cenário Simulado
Foram simuladas epidemias usando o modelo SIR sobre três redes com topologias distintas:

* Erdős–Rényi (ER): conexões aleatórias com grau homogêneo

* Barabási–Albert (BA): rede com hubs altamente conectados

* Watts–Strogatz (WS): rede de mundo pequeno, com alta transitividade e atalhos




Cada rede foi submetida a remoções progressivas de até 50% das conexões, segundo três estratégias:

1. Remoção aleatória: representa distanciamento social generalizado, sem foco estratégico;

2. Remoção por betweenness centrality: remove as arestas mais estruturais, que atuam como “pontes”;

3. Remoção por grau médio dos vértices: remove conexões entre indivíduos de alta conectividade.



# Principais Resultados
🔴 Rede Barabási–Albert (BA)
Apresentou a maior sensibilidade à remoção de links, devido à presença de hubs.

A remoção orientada por betweenness foi a mais eficaz, mesmo com baixos percentuais (~10%).

A remoção aleatória teve efeito mais modesto, exigindo remoção acima de 30% para conter a epidemia.

🔵 Rede Erdős–Rényi (ER)
Devido à estrutura homogênea, todas as estratégias tiveram impacto relativamente parecido.

A redução da propagação foi progressiva e linear, independentemente da centralidade.

🟢 Rede Watts–Strogatz (WS)
A remoção por betweenness reduziu significativamente a epidemia ao eliminar atalhos cruciais entre comunidades.

A remoção por grau médio teve efeito moderado.

A remoção aleatória foi menos eficaz, exigindo cortes mais extensos para impacto visível.

#Conclusão Geral
Estratégias inteligentes e direcionadas de distanciamento social (com base em centralidade) são mais eficazes do que a remoção aleatória, especialmente em redes com heterogeneidade estrutural (como BA).

A eficácia da contenção depende fortemente da topologia da rede:

Em redes com hubs, interromper conexões estratégicas colapsa rapidamente a disseminação.

Em redes homogêneas (ER), o efeito é mais difuso, mas cumulativo.

Em redes WS, atalhos e agrupamentos locais tornam certas conexões especialmente críticas.